In [ ]:
# Generación del archivo requirements.txt
!pip freeze > ../requirements.txt

In [ ]:
from time import sleep
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar
import json
import sys
from datetime import datetime
sys.path.append("../")
import src.soporte_extraccion as se

# Extracción de URLs con Selenium y Beautiful Soup

In [ ]:
# Creo el driver que utiliza Selenium
driver = webdriver.Chrome()

# Generó por defecto el tiempo de espera para encontrar el elemento que le pido
driver.implicitly_wait(10)

# Abro la página de la que vamos a extraer los datos
url_facua = "https://super.facua.org/"
driver.get(url_facua)

# Maximizo la página para mostrar todos los elementos. Esto se hace por si tiene responsive y esconde elementos
driver.maximize_window()
try:
    # Hago una pausa para que cargue el html de la página y asegurarme encontrar el elemento que le pido posteriormente
    sleep(1)

    # Acepto las cookies para que no molesten el resto del scrap
    driver.find_element("css selector", "#rcc-confirm-button").click()

    # Extracción de urls con Selenium
    diccionario_urls = se.obtener_lista_url(driver)
    print("Primera lista de URLs obtenida")

    # Extracción de urls de categorias con Selenium
    diccionario_categorias = se.obtener_categorias(diccionario_urls, driver)
    print("Segunda lista de URLs obtenida")

    # Cierro el driver de Selenium porque ya hemos extraido todo lo necesario
    driver.close()
    
    # Extracción de urls de los productos con Beautiful Soup
    diccionario_subcategorias = se.obtener_subcategorias(diccionario_categorias)
    print("Tercera lista de URLs obtenida")
    print("URLs extraidas")
    
except NoSuchElementException as error:
    # En caso de error muestra este mensaje
    print(f"Error al encontrar el elemento: {error.msg}")

In [8]:
# Guardo el diccionario creado en un archivo json para no tener que ejecutar de nuevo el código anterior
with open('../datos/raw/urls.json', 'w') as jf:
    json.dump(diccionario_subcategorias, jf, ensure_ascii=False, indent=2)

In [2]:
# Compruebo que se ha guardado correctamente el json
with open('../datos/raw/urls.json', 'r') as jf: 
    diccionario_urls2 = json.load(jf)

In [ ]:
# Extraigo la información de los precios historicos de las tablas con una función asíncrona
df_precios_historicos = await se.extraer_informacion(diccionario_urls2)

In [13]:
# Guardo el dataframe con todos los registros en un csv
df_precios_historicos.to_csv(f"../datos/raw/precios_historicos_{datetime.now().date()}.csv")